In [ ]:
#Import Libraries
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
import matplotlib.pyplot as plt

!pip install pandasql
import os
from datetime import datetime
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_error

import math
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pandasql as ps
from sqlite3 import connect
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

conn=connect(':memory:')

In [ ]:
#SET PATHS
main="/content/gdrive/MyDrive/"
raw= main + '_raw/'
aux= raw + 'AUX/'
clean= main + '_clean/'

# 01. Import Daily  Price Data

In [ ]:
file="DailyStockPrice.csv"


price_df=pd.read_csv(raw + file)
# Parse the 'date' column to a datetime format
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')

# Filter to only keep data from 2018 onwards
price_df = price_df[(price_df['date'] >= "2015-01-01")]

# Parse Relevant Variables, only keep distinct rows
price_df = price_df[['ticker', 'date', 'adj_close', 'adj_volume']].drop_duplicates()
print(price_df)


         ticker       date  adj_close  adj_volume
0          MODN 2017-03-13      10.25     73278.0
1          MODN 2017-03-14      10.25     98476.0
2          MODN 2017-03-15      10.30     57470.0
3          MODN 2017-03-16      10.45     78305.0
4          MODN 2017-03-17      10.50    122109.0
...         ...        ...        ...         ...
46006631   MODN 2017-03-06      10.10     91871.0
46006632   MODN 2017-03-07       9.90    154676.0
46006633   MODN 2017-03-08       9.85     81438.0
46006634   MODN 2017-03-09      10.00    148842.0
46006635   MODN 2017-03-10      10.05     80926.0

[20693885 rows x 4 columns]


# 02. Import Zachs Master Data as Ticker Map

In [ ]:


file= "ZACKS_MT_2.csv"

# Read the CSV file into a DataFrame 'map'
map = pd.read_csv(aux + file)



# Keep distinct rows based on 'ticker', 'exchange', 'asset_type', 'comp_type'
map = map[['ticker', 'exchange', 'asset_type', 'comp_type']].drop_duplicates()

# Merge 'price_df' and 'map' DataFrames on the 'ticker' column
price_df_xmap = pd.merge(price_df, map, on='ticker', how='left')

# Convert to string type for filtering
price_df_xmap['exchange'] = price_df_xmap['exchange'].astype(str)
price_df_xmap['asset_type'] = price_df_xmap['asset_type'].astype(str)
price_df_xmap['comp_type'] = price_df_xmap['comp_type'].astype(str)

# Keep only rows where 'exchange' is either 'NYSE', 'AMEX', or 'NASDAQ'
relevant_exchanges = ['NYSE', 'AMEX', 'NASDAQ']
price_df_xmap = price_df_xmap[price_df_xmap['exchange'].isin(relevant_exchanges)]

# Keep only US Based common stocks where 'asset_type' is 'COM'
price_df_xmap = price_df_xmap[price_df_xmap['asset_type'] == 'COM']

# Keep only industrial stocks where 'comp_type' is '1.0'
price_df_xmap = price_df_xmap[price_df_xmap['comp_type'] == '1.0']

<ipython-input-4-dc53322f1d36>:4: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  map = pd.read_csv(aux + file)


# 03. Calculate Daily Returns

In [ ]:
price_df = price_df_xmap

# Sort by 'ticker' and 'date', so stocks are listed consecutively
price_df = price_df.sort_values(['ticker', 'date'], ascending=True)

# Create a 'stock checker' column, shifted by one for previous day's ticker
#price_df['stock checker'] = price_df['ticker'].shift(1)

# Only keep rows where 'ticker' matches 'stock checker', i.e., we have prior day data for the same stock
#price_df = price_df[price_df['ticker'] == price_df['stock checker']]

# Calculate the 'daily_return' column
price_df['adj_close'] = pd.to_numeric(price_df['adj_close'], errors='coerce')
price_df['adj_close_prev_day'] = price_df.groupby('ticker')['adj_close'].shift(1)
price_df['daily_return'] = ((price_df['adj_close'] - price_df['adj_close_prev_day']) / price_df['adj_close_prev_day']) * 100

# Parse relevant variables and remove duplicates
price_df = price_df[['ticker', 'date', 'adj_close', 'daily_return', 'exchange']].drop_duplicates()

In [ ]:
price_df

,ticker,date,adj_close,daily_return,exchange
20684258,A,2015-01-02,37.837363,NaN,NYSE
20684257,A,2015-01-05,37.128379,-1.873767,NYSE
17304327,A,2015-01-06,36.549997,-1.557789,NYSE
20684256,A,2015-01-07,37.035092,1.327208,NYSE
20684255,A,2015-01-08,38.145211,2.997481,NYSE
...,...,...,...,...,...
7946248,ZYXI,2023-06-16,9.630000,1.689546,NASDAQ
2926385,ZYXI,2023-06-20,9.790000,1.661475,NASDAQ
4499090,ZYXI,2023-06-21,9.690000,-1.021450,NASDAQ
13335400,ZYXI,2023-06-22,9.460000,-2.373581,NASDAQ


# 04. Import Raw Fundamental Files

In [ ]:

# Read data from CSV files
z_fc2 = pd.read_csv(aux + "ZACKS_FC_2.csv")
z_mktv2 = pd.read_csv(aux + "ZACKS_MKTV_2.csv")
z_fr2 = pd.read_csv(aux + "ZACKS_FR_2.csv")

# Keep distinct rows based on specified columns
z_mktv2 = z_mktv2.drop_duplicates()

z_fc2 = z_fc2.drop_duplicates()

z_fr2 = z_fr2.drop_duplicates()


# Only keep quarterly data
z_fr2 = z_fr2[z_fr2['per_type'] == "Q"]
z_fc2 = z_fc2[z_fc2['per_type'] == "Q"]
z_mktv2 = z_mktv2[z_mktv2['per_type'] == "Q"]

# Standardize 'per_end_date' and 'ticker' columns
for df in [z_fr2, z_fc2, z_mktv2]:
    df['per_end_date'] = pd.to_datetime(df['per_end_date'], format='%Y-%m-%d')
    df['ticker'] = df['ticker'].astype(str)

# Merge the dataframes on 'ticker' and 'per_end_date'
fundamentals = z_fc2.merge(z_fr2[['ticker', 'per_end_date', 'book_val_per_share']], how='left', on=['ticker', 'per_end_date'])
fundamentals = fundamentals.merge(z_mktv2[['ticker', 'per_end_date', 'mkt_val']], how='left', on=['ticker', 'per_end_date'])

# Keep only rows where 'exchange' is either "NYSE", "AMEX", or "NASDAQ"
fundamentals = fundamentals[fundamentals['exchange'].isin(["NYSE", "AMEX", "NASDAQ"])]

<ipython-input-7-26c1ec22c030>:2: DtypeWarning: Columns (4,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  z_fc2 = pd.read_csv(aux + "ZACKS_FC_2.csv")


# ---------------------------------------------------

In [ ]:
for i in fundamentals.columns:
  print(i)

In [ ]:
'''Book Equity. Book equity is constructed from Compustat data or collected from the Moody’s Industrial,
Financial, and Utilities manuals. BE is the book value of stockholders’ equity, plus balance sheet deferred
taxes and investment tax credit (if available), minus the book value of preferred stock.
Depending on availability, we use the redemption, liquidation, or par value (in that order) to estimate the book value of preferred stock. Stockholders’ equity is the value reported by Moody’s or Compustat, if it is available. If not,
we measure stockholders’ equity as the book value of common equity plus the par value of preferred stock,
or the book value of assets minus total liabilities (in that order). '''

'Book Equity. Book equity is constructed from Compustat data or collected from the Moody’s Industrial,\nFinancial, and Utilities manuals. BE is the book value of stockholders’ equity, plus balance sheet deferred\ntaxes and investment tax credit (if available), minus the book value of preferred stock.\nDepending on availability, we use the redemption, liquidation, or par value (in that order) to estimate the book value of preferred stock. Stockholders’ equity is the value reported by Moody’s or Compustat, if it is available. If not,\nwe measure stockholders’ equity as the book value of common equity plus the par value of preferred stock,\nor the book value of assets minus total liabilities (in that order). '

In [ ]:
#set tax credits and preferred equity to 0 where missing

fundamentals['def_tax_asset_curr'] = fundamentals['def_tax_asset_curr'].fillna(0)
fundamentals['def_tax_asset_lterm'] = fundamentals['def_tax_asset_lterm'].fillna(0)
fundamentals['tot_pref_stock'] = fundamentals['tot_pref_stock'].fillna(0)

<ipython-input-10-044cc08c72fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fundamentals['def_tax_asset_curr'] = fundamentals['def_tax_asset_curr'].fillna(0)


In [ ]:
#Calculate book equity as per FF definition

fundamentals.to_sql('fundamentals', conn, if_exists='replace')

query='''
        SELECT DISTINCT *,
        COALESCE(tot_share_holder_equity, tot_asset-tot_liab) + def_tax_asset_curr + def_tax_asset_lterm - tot_pref_stock as book_value
        FROM fundamentals
        '''
fundamentals_book_value=pd.read_sql(query, conn)

In [ ]:
fundamentals=fundamentals_book_value

In [ ]:
fundamentals['book_value'].isnull().sum()

2053

# 05. Pull Relevant Variables From Fundamentals

**Market Equity:**

In [ ]:
cleaned_returns = price_df.copy()

# Convert 'date' to datetime format and create a 'quarter' column
cleaned_returns['date'] = pd.to_datetime(cleaned_returns['date'])
cleaned_returns['quarter'] = cleaned_returns['date'].dt.to_period('Q')

# Create a 'last_quarter' column by subtracting one quarter from the 'date' column
cleaned_returns['last_quarter'] = (cleaned_returns['date'] - pd.DateOffset(months=3)).dt.to_period('Q')

# Convert 'per_end_date' to datetime format and create a 'quarter' column in fundamentals dataframe
fundamentals['per_end_date'] = pd.to_datetime(fundamentals['per_end_date'])
fundamentals['quarter'] = fundamentals['per_end_date'].dt.to_period('Q')

# Convert 'quarter' columns to string format for joining
cleaned_returns['last_quarter'] = cleaned_returns['last_quarter'].astype(str)
cleaned_returns['quarter'] = cleaned_returns['quarter'].astype(str)
fundamentals['quarter'] = fundamentals['quarter'].astype(str)

# Convert 'ticker' columns to string format
cleaned_returns['ticker'] = cleaned_returns['ticker'].astype(str)
fundamentals['ticker'] = fundamentals['ticker'].astype(str)

# Sort fundamentals dataframe by 'mkt_val'
fundamentals = fundamentals.sort_values(by='mkt_val', ascending=False)

# Merge cleaned_returns and fundamentals dataframes
cleaned_returns = cleaned_returns.merge(fundamentals[['ticker', 'quarter', 'mkt_val', 'comm_shares_out']], how='left',
                                        left_on=['ticker', 'last_quarter'],
                                        right_on=['ticker', 'quarter'], suffixes=('', '_y'))

# Rename 'mkt_val' to 'mkt_cap'
cleaned_returns.rename(columns={'mkt_val': 'mkt_cap_old'}, inplace=True)

# Remove unnecessary columns from the merged dataframe
cleaned_returns.drop(columns=['quarter_y'], inplace=True)


**Calculate ME as per FF: PRICE * SHARES OUTSTANDING**

In [ ]:
cleaned_returns['mkt_cap']=cleaned_returns['comm_shares_out']*cleaned_returns['adj_close']

**Pull Book Value From The End of Each Year to Calculate B/M Ratio**

In [ ]:
# To create a DataFrame similar to 'fundamentals_be' using the 'fundamentals' DataFrame:
fundamentals_be = fundamentals[['ticker', 'per_end_date', 'book_value']].copy()

# Convert 'per_end_date' to datetime format and extract 'quarter' and 'month'
fundamentals_be['per_end_date'] = pd.to_datetime(fundamentals_be['per_end_date'])
fundamentals_be['quarter'] = fundamentals_be['per_end_date'].dt.to_period('Q')
fundamentals_be['month'] = fundamentals_be['per_end_date'].dt.month

# Filter the DataFrame to keep only the entries where 'month' is 12
fundamentals_be = fundamentals_be[fundamentals_be['month'] == 12]

# Rename 'per_end_date' to 'date'
fundamentals_be.rename(columns={'per_end_date': 'date'}, inplace=True)
fundamentals_be

,ticker,date,book_value,quarter,month
302,AAPL,2021-12-31,71932.000,2021Q4,12
67880,MSFT,2021-12-31,160010.000,2021Q4,12
298,AAPL,2020-12-31,66224.000,2020Q4,12
306,AAPL,2022-12-31,56727.000,2022Q4,12
45070,GOOGL,2021-12-31,252919.000,2021Q4,12
...,...,...,...,...,...
113924,ZYXI,2018-12-31,10.016,2018Q4,12
113928,ZYXI,2019-12-31,20.166,2019Q4,12
113932,ZYXI,2020-12-31,57.421,2020Q4,12
113936,ZYXI,2021-12-31,74.636,2021Q4,12


In [ ]:
#add reference date for matching
fundamentals_be['year']=fundamentals_be['date'].dt.year

fundamentals_be['year']=pd.to_numeric(fundamentals_be['year'], errors='coerce')

#ADD REFERENCE DATE FOR MATCHING (6/1 of next calendar year)
fundamentals_be['reference_date']=fundamentals_be['year']+1
fundamentals_be['reference_date']=fundamentals_be['reference_date'].apply(str)
fundamentals_be['reference_date'] = fundamentals_be['reference_date'].apply(lambda x: x + "-06-01")
fundamentals_be['reference_date']=pd.to_datetime(fundamentals_be['reference_date'])
fundamentals_be.drop('year', inplace=True, axis=1)


In [ ]:
fundamentals_be

,ticker,date,book_value,quarter,month,reference_date
302,AAPL,2021-12-31,71932.000,2021Q4,12,2022-06-01
67880,MSFT,2021-12-31,160010.000,2021Q4,12,2022-06-01
298,AAPL,2020-12-31,66224.000,2020Q4,12,2021-06-01
306,AAPL,2022-12-31,56727.000,2022Q4,12,2023-06-01
45070,GOOGL,2021-12-31,252919.000,2021Q4,12,2022-06-01
...,...,...,...,...,...,...
113924,ZYXI,2018-12-31,10.016,2018Q4,12,2019-06-01
113928,ZYXI,2019-12-31,20.166,2019Q4,12,2020-06-01
113932,ZYXI,2020-12-31,57.421,2020Q4,12,2021-06-01
113936,ZYXI,2021-12-31,74.636,2021Q4,12,2022-06-01


Pull Total Asset Value From End of Year to Calculate Investment

In [ ]:
fundamentals_inv=fundamentals[['ticker', 'per_end_date', 'tot_asset']].copy()

fundamentals_inv['per_end_date'] = pd.to_datetime(fundamentals_inv['per_end_date'])
fundamentals_inv['quarter'] = fundamentals_inv['per_end_date'].dt.to_period('Q')
fundamentals_inv['month'] = fundamentals_inv['per_end_date'].dt.month


fundamentals_inv = fundamentals_inv[fundamentals_inv['month'] == 12]

# Rename 'per_end_date' to 'date'
fundamentals_inv.rename(columns={'per_end_date': 'date'}, inplace=True)

fundamentals_inv.sort_values(by=['ticker', 'date'], inplace=True)
fundamentals_inv

,ticker,date,tot_asset,quarter,month
30,AA,2017-12-31,17447.000,2017Q4,12
34,AA,2018-12-31,16132.000,2018Q4,12
38,AA,2019-12-31,14631.000,2019Q4,12
42,AA,2020-12-31,14860.000,2020Q4,12
46,AA,2021-12-31,15025.000,2021Q4,12
...,...,...,...,...,...
113932,ZYXI,2020-12-31,72.175,2020Q4,12
113936,ZYXI,2021-12-31,133.274,2021Q4,12
113940,ZYXI,2022-12-31,116.466,2022Q4,12
69061,nan,2021-12-31,142.703,2021Q4,12


Calculate investment as (assets t-1 -assets t-2)/[assets t-2]

In [ ]:
#shift back investment by ticker
fundamentals_inv.sort_values(by=['ticker', 'date'], inplace=True)
fundamentals_inv['prev_year_tot_asset'] = fundamentals_inv.groupby('ticker')['tot_asset'].shift(1)

fundamentals_inv['inv']=(fundamentals_inv['tot_asset']-fundamentals_inv['prev_year_tot_asset'])/(fundamentals_inv['tot_asset'])

In [ ]:
# Add reference date for matching
fundamentals_inv['year'] = fundamentals_inv['date'].dt.year

fundamentals_inv['year'] = pd.to_numeric(fundamentals_inv['year'], errors='coerce')

# Add reference date for matching (6/1 of next calendar year)
fundamentals_inv['reference_date'] = fundamentals_inv['year']+1
fundamentals_inv['reference_date'] = fundamentals_inv['reference_date'].apply(str)
fundamentals_inv['reference_date'] = fundamentals_inv['reference_date'].apply(lambda x: x + "-06-01")
fundamentals_inv['reference_date'] = pd.to_datetime(fundamentals_inv['reference_date'])
fundamentals_inv.drop('year', inplace=True, axis=1)

# 06a. Construct the Stocks Data

In [ ]:
# Construct main_stocks_df
main_stocks_df = cleaned_returns[['ticker', 'exchange', 'date', 'daily_return', 'quarter', 'mkt_cap', 'adj_close', 'comm_shares_out']].copy()

# Rename column
main_stocks_df.rename(columns={'daily_return': 'ret'}, inplace=True)

# Convert 'date' to datetime
main_stocks_df['date'] = pd.to_datetime(main_stocks_df['date'])

# Add 'reference_date' column
main_stocks_df['reference_date'] = main_stocks_df['date'].apply(
    lambda dt: pd.Timestamp(year=dt.year - 1, month=6, day=1) if dt.month < 6 else pd.Timestamp(year=dt.year, month=6, day=1)
)


# 06b. Merge B/M Into Stocks Data

In [ ]:
# Convert 'reference_date' columns to datetime
fundamentals_be['reference_date'] = pd.to_datetime(fundamentals_be['reference_date'])
main_stocks_df['reference_date'] = pd.to_datetime(main_stocks_df['reference_date'])

# Merge main_stocks_df and fundamentals_be dataframes on 'ticker' and 'reference_date'
main_stocks_df = main_stocks_df.merge(fundamentals_be[['ticker', 'reference_date', 'book_value']],
                                      on=['ticker', 'reference_date'],
                                      how='left')


# 06c. Add ME From End of Year

In [ ]:
# Filter rows with 'date' in December
stocks_me = main_stocks_df[main_stocks_df['date'].dt.month == 12].copy()

# Select distinct rows based on 'ticker' and 'quarter'
stocks_me = stocks_me[['ticker', 'date', 'mkt_cap', 'quarter']].drop_duplicates()

# Convert the 'date' column to datetime format
stocks_me['date'] = pd.to_datetime(stocks_me['date'])

# Extract year and month
stocks_me['year'] = stocks_me['date'].dt.year
stocks_me['month'] = stocks_me['date'].dt.month

# Group by ticker, year, and month and select the last observation
stocks_me_last_obs = stocks_me.sort_values('date').groupby(['ticker', 'year', 'month']).last().reset_index()
stocks_me=stocks_me_last_obs


# Extract year and quarter from 'quarter' column
stocks_me[['year', 'quarter']] = stocks_me['quarter'].str.split('Q', expand=True)

# Convert 'year' to numeric
stocks_me['year'] = stocks_me['year'].astype(int)

# Create 'reference_date' for next year
stocks_me['reference_date'] = pd.to_datetime((stocks_me['year'] + 1).astype(str) + "-06-01")

# Rename 'mkt_cap' to 'mkt_equity'
stocks_me.rename(columns={'mkt_cap': 'mkt_equity'}, inplace=True)

# Drop unnecessary columns
stocks_me.drop(['year', 'quarter'], axis=1, inplace=True)

# Merge 'main_stocks_df' and 'stocks_me' on 'ticker' and 'reference_date'
main_stocks_df = pd.merge(main_stocks_df, stocks_me, on=['ticker', 'reference_date'], how='left')



#06d. Calculate BM Ratio: BE/ME

In [ ]:
main_stocks_df['bm_ratio']=main_stocks_df['book_value']/main_stocks_df['mkt_equity']

negative_values = main_stocks_df[main_stocks_df['bm_ratio'] < 0]
count_negative = len(negative_values)
print("Number of negative values in 'bm_ratio':", count_negative)

Number of negative values in 'bm_ratio': 236860


# 06e. Merge Investments Into Stocks Data

In [ ]:
# Convert 'reference_date' columns to datetime
fundamentals_inv['reference_date'] = pd.to_datetime(fundamentals_inv['reference_date'])
main_stocks_df['reference_date'] = pd.to_datetime(main_stocks_df['reference_date'])

# Merge main_stocks_df and fundamentals_be dataframes on 'ticker' and 'reference_date'
main_stocks_df = main_stocks_df.merge(fundamentals_inv[['ticker', 'reference_date', 'inv']],
                                      on=['ticker', 'reference_date'],
                                      how='left')

# 06f. Merge OP into Stocks Data

In [ ]:
fundamentals_op=pd.read_csv(clean+"op.csv")

In [ ]:
# Convert 'reference_date' columns to datetime
fundamentals_op['reference_date'] = pd.to_datetime(fundamentals_op['reference_date'])
main_stocks_df['reference_date'] = pd.to_datetime(main_stocks_df['reference_date'])

# Merge main_stocks_df and fundamentals_be dataframes on 'ticker' and 'reference_date'
main_stocks_df = main_stocks_df.merge(fundamentals_op[['ticker', 'reference_date', 'op']],
                                      on=['ticker', 'reference_date'],
                                      how='left')

In [ ]:
main_stocks_df['op'].isnull().sum()

2698103

# 07. Size Sorts

In [ ]:
main_stocks_df_copy=main_stocks_df.copy()

In [ ]:
main_stocks_df.drop('date_y', axis=1, inplace=True)
main_stocks_df.rename(columns={'date_x': 'date'}, inplace=True)


# Pre-processing: replacing infinite values with NaN and dropping NaN values
main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_stocks_df.dropna(inplace=True)

# Extract quarter from the date
main_stocks_df['date'] = pd.to_datetime(main_stocks_df['date'])
main_stocks_df['quarter'] = main_stocks_df['date'].dt.quarter

# Filter data for June and NYSE to calculate the median market cap
size_breakpoints = main_stocks_df[(main_stocks_df['date'].dt.month == 6) & (main_stocks_df['exchange'] == "NYSE")].copy()
size_breakpoints=size_breakpoints[['ticker', 'reference_date', 'mkt_cap', 'date']].drop_duplicates()

# Convert the 'date' column to datetime format
size_breakpoints['date'] = pd.to_datetime(size_breakpoints['date'])

# Group by ticker and reference_date and select the last observation
size_breakpoints_last_obs = size_breakpoints.sort_values('date').groupby(['ticker', 'reference_date']).last().reset_index()


size_breakpoints=size_breakpoints_last_obs
size_median = size_breakpoints.groupby('reference_date')['mkt_cap'].median().reset_index()
size_median.columns = ['reference_date', 'size_median']
size_median

,reference_date,size_median
0,2019-06-01,3063.278360
1,2020-06-01,2595.085451
2,2021-06-01,4624.327000
3,2022-06-01,2968.458961
4,2023-06-01,3423.495905


In [ ]:
size_breakpoints_last_obs

,ticker,reference_date,mkt_cap,date
0,AA,2019-06-01,4282.343218,2019-06-28
1,AA,2020-06-01,2060.365625,2020-06-30
2,AA,2021-06-01,6782.289119,2021-06-30
3,AA,2022-06-01,8330.420258,2022-06-30
4,AA,2023-06-01,5858.688840,2023-06-23
...,...,...,...,...
3936,ZWS,2019-06-01,3076.468853,2019-06-28
3937,ZWS,2020-06-01,3406.243100,2020-06-30
3938,ZWS,2021-06-01,5898.063336,2021-06-30
3939,ZWS,2022-06-01,3389.706589,2022-06-30


In [ ]:
# Create new dataframe for all stocks in June
size_sorts = main_stocks_df[main_stocks_df['date'].dt.month == 6].copy()
size_sorts=size_sorts[['ticker', 'reference_date', 'mkt_cap', 'date']].drop_duplicates()

size_sorts_last_obs = size_sorts.sort_values('date').groupby(['ticker', 'reference_date']).last().reset_index()
size_sorts=size_sorts_last_obs

# Merge the median size into size_sorts
size_sorts = size_sorts.merge(size_median, on='reference_date', how='left')

#Assign to Relevant Size Portfolio
size_sorts['size_portfolio'] = np.where(size_sorts['mkt_cap'] > size_sorts['size_median'], 'B', 'S')

# Merge size_sorts data back into main_stocks_df
main_stocks_df = main_stocks_df.merge(size_sorts, on=['ticker', 'reference_date'], how='left')

main_stocks_df.drop(['date_y', 'mkt_cap_y'], axis=1, inplace=True)
main_stocks_df.rename(columns={'date_x':'date', 'mkt_cap_x':'mkt_cap'}, inplace=True)




# 08. Investment Sorts

In [ ]:
main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)


# Filter data for June and NYSE
inv_breakpoints = main_stocks_df[(main_stocks_df['date'].dt.month == 6) & (main_stocks_df['exchange'] == "NYSE")].copy()

# Get distinct values of ticker, reference_date, and bm_ratio
inv_breakpoints = inv_breakpoints[['ticker', 'reference_date', 'inv']].drop_duplicates()

# Calculate the 30th and 70th quantiles of bm_ratio for each reference_date
inv_breakpoints['q30'] = inv_breakpoints.groupby('reference_date')['inv'].transform(lambda x: x.quantile(0.3))
inv_breakpoints['q70'] = inv_breakpoints.groupby('reference_date')['inv'].transform(lambda x: x.quantile(0.7))

inv_breakpoints = inv_breakpoints[['reference_date', 'q30', 'q70']].drop_duplicates()
inv_breakpoints

,reference_date,q30,q70
0,2019-06-01,-0.013285,0.085353
251,2020-06-01,0.013925,0.130797
503,2021-06-01,-0.005824,0.103494
756,2022-06-01,0.020124,0.203810
1007,2023-06-01,-0.023397,0.076868


In [ ]:
# Create new dataframe for all stocks in June
inv_sorts = main_stocks_df[main_stocks_df['date'].dt.month == 6].copy()
inv_sorts=inv_sorts[['ticker', 'reference_date', 'inv']].drop_duplicates()

#Merge in breakpoints
inv_sorts=inv_sorts.merge(inv_breakpoints, on='reference_date', how='left')

inv_breakpoints=inv_sorts

# Classify stocks as 'L', 'M', or 'H' based on the quantiles
inv_breakpoints['inv_portfolio'] = np.where(inv_breakpoints['inv'] > inv_breakpoints['q70'], 'H',
                                                np.where(inv_breakpoints['inv'] > inv_breakpoints['q30'], 'M', 'L'))


# Merge value_breakpoints data back into main_stocks_df
main_stocks_df = main_stocks_df.merge(inv_breakpoints[['ticker', 'reference_date', 'inv_portfolio']],
                                      on=['ticker', 'reference_date'], how='left')

In [ ]:
main_stocks_df.to_csv()

# 08. Value Sorts

In [ ]:
main_stocks_df_copy=main_stocks_df.copy()

In [ ]:
# Prepare data- drop stocks with negative BE
main_stocks_df = main_stocks_df[main_stocks_df['bm_ratio'] >= 0]

main_stocks_df.replace([np.inf, -np.inf], np.nan, inplace=True)
main_stocks_df.dropna()

# Extract quarter from the date
main_stocks_df['date'] = pd.to_datetime(main_stocks_df['date'])
main_stocks_df['quarter'] = main_stocks_df['date'].dt.quarter

# Filter data for June and NYSE
value_breakpoints = main_stocks_df[(main_stocks_df['date'].dt.month == 6) & (main_stocks_df['exchange'] == "NYSE")].copy()

# Get distinct values of ticker, reference_date, and bm_ratio
value_breakpoints = value_breakpoints[['ticker', 'reference_date', 'bm_ratio']].drop_duplicates()

# Calculate the 30th and 70th quantiles of bm_ratio for each reference_date
value_breakpoints['q30'] = value_breakpoints.groupby('reference_date')['bm_ratio'].transform(lambda x: x.quantile(0.3))
value_breakpoints['q70'] = value_breakpoints.groupby('reference_date')['bm_ratio'].transform(lambda x: x.quantile(0.7))

value_breakpoints=value_breakpoints[['reference_date', 'q30', 'q70']].drop_duplicates()
value_breakpoints

In [ ]:
# Create new dataframe for all stocks in June
value_sorts = main_stocks_df[main_stocks_df['date'].dt.month == 6].copy()
value_sorts=value_sorts[['ticker', 'reference_date', 'bm_ratio']].drop_duplicates()

#Merge in breakpoints
value_sorts=value_sorts.merge(value_breakpoints, on='reference_date', how='left')

value_breakpoints=value_sorts

# Classify stocks as 'L', 'M', or 'H' based on the quantiles
value_breakpoints['value_portfolio'] = np.where(value_breakpoints['bm_ratio'] > value_breakpoints['q70'], 'H',
                                                np.where(value_breakpoints['bm_ratio'] > value_breakpoints['q30'], 'M', 'L'))

# Merge value_breakpoints data back into main_stocks_df
main_stocks_df=main_stocks_df_copy
main_stocks_df = main_stocks_df.merge(value_breakpoints[['ticker', 'reference_date', 'value_portfolio']],
                                      on=['ticker', 'reference_date'], how='left')

In [ ]:
negative_values = main_stocks_df[main_stocks_df['bm_ratio'] < 0]
count_negative = len(negative_values)
print("Number of negative values in 'bm_ratio':", count_negative)

# OP Sorts

In [ ]:

op_breakpoints = main_stocks_df[(main_stocks_df['date'].dt.month == 6) & (main_stocks_df['exchange'] == "NYSE")].copy()
# Get distinct values of ticker, reference_date, and bm_ratio
op_breakpoints = op_breakpoints[['ticker', 'reference_date', 'op']].drop_duplicates()

# Calculate the 30th and 70th quantiles of bm_ratio for each reference_date
op_breakpoints['q30'] = op_breakpoints.groupby('reference_date')['op'].transform(lambda x: x.quantile(0.3))
op_breakpoints['q70'] = op_breakpoints.groupby('reference_date')['op'].transform(lambda x: x.quantile(0.7))

op_breakpoints = op_breakpoints[['reference_date', 'q30', 'q70']].drop_duplicates()
op_breakpoints


In [ ]:
# Create new dataframe for all stocks in June
op_sorts = main_stocks_df[main_stocks_df['date'].dt.month == 6].copy()
op_sorts=op_sorts[['ticker', 'reference_date', 'op']].drop_duplicates()

op_sorts=op_sorts.merge(op_breakpoints, on='reference_date', how='left')

op_breakpoints=op_sorts

# Classify stocks as 'L', 'M', or 'H' based on the quantiles
op_breakpoints['op_portfolio'] = np.where(op_breakpoints['op'] > op_breakpoints['q70'], 'H',
                                                np.where(op_breakpoints['op'] > op_breakpoints['q30'], 'M', 'L'))

# Merge op_breakpoints data back into main_stocks_df

main_stocks_df = main_stocks_df.merge(op_breakpoints[['ticker', 'reference_date', 'op_portfolio']],
                                      on=['ticker', 'reference_date'], how='left')


# Export Data

In [ ]:
clean= main + '_clean/'
main_stocks_df.to_csv(clean + "FF5 Dataload 07_23.csv")


In [ ]:
len(main_stocks_df)

In [ ]:
main

In [ ]:
f=main_stocks_df[main_stocks_df['inv'].isnull()]